In [349]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [350]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [351]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 32

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[ 1.0515, -0.6268, -1.5157,  1.2130, -0.8868, -1.1206,  0.7028, -0.8795,
         -0.5306,  1.7867,  0.1185, -1.4041, -0.5725, -0.0348, -0.6039,  0.5826,
         -0.4816, -1.6205, -1.3969, -0.3553, -1.3570, -0.7840, -0.2518,  0.4940,
         -0.2615, -0.0922,  0.0173,  0.2895,  0.3179, -1.4148, -0.7734,  0.6145],
        [ 2.0033,  2.8390, -0.8576, -1.4089, -0.4146, -0.0476,  0.1956, -0.0263,
         -0.4050, -0.0279,  1.0718,  0.5244,  2.5485, -1.9113, -0.5397,  0.4045,
         -0.8233, -1.7529, -0.3423,  0.2473, -0.2098, -0.8016, -0.5671,  0.8350,
         -2.0431, -0.1595,  1.3528, -0.1737,  2.2997, -0.7037, -0.2881,  0.0678]],
       grad_fn=<EmbeddingBackward0>)


In [352]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # non-sos,eos,pad

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [353]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 5,  0,  6,  9,  3,  2,  7,  4,  5, 10],
        [ 4,  3,  6,  5,  4,  5, 10, 10, 10, 10],
        [ 7,  0,  3,  9, 10, 10, 10, 10, 10, 10],
        [ 2,  9,  7,  4,  3, 10, 10, 10, 10, 10],
        [ 1,  9,  5,  1, 10, 10, 10, 10, 10, 10],
        [ 0,  5, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  6, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  2,  0,  9, 10, 10, 10, 10, 10, 10],
        [ 3,  1,  4,  8, 10, 10, 10, 10, 10, 10],
        [ 7,  5,  6,  2,  4,  6,  6,  4,  7,  5],
        [ 3,  1,  5,  5,  5,  5,  0,  8,  6,  3],
        [ 2,  7, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 0,  0,  2, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  4,  9,  1,  7, 10, 10, 10, 10, 10],
        [ 3,  8,  8,  1,  2,  9,  1,  1,  1,  5],
        [ 5,  6, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  4,  9,  5,  2,  4,  0,  7,  7,  1],
        [ 3,  4,  7,  5,  0,  2,  7, 10, 10, 10],
        [ 5,  5,  4,  6,  7,  1,  5,  9,  8,  9],
        [ 7,  6,  5,  6,  3, 10, 10, 10, 10, 10],


In [354]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    dropout = 0.0,
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [355]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    """
    """
    model.eval()

    with torch.no_grad():
        ...

In [356]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 32/32 [00:00<00:00, 321.74it/s]

[3.7824424743652343, 2.3522406171798704, 2.131438836288452, 1.6965927942276, 1.9103202781677247, 1.8295921804428101, 1.5049847660064697, 1.4966392545700073, 1.4042596161842347, 1.2617768876075746]
[0.0047, 0.0128, 0.017, 0.0251, 0.0237, 0.0249, 0.0328, 0.0311, 0.0367, 0.0416]
[0.5054715950473416, 0.5583229972274916, 0.5678959457488698, 0.5818198395331875, 0.5757848921911422, 0.5790240349599417, 0.5898676727909011, 0.5898056485660212, 0.5934814275309541, 0.5984837439860038]

[3.4687255401611328, 1.7385754241943359, 2.039300661087036, 2.2033851146698, 1.5330677003860473, 1.7821049776077271, 1.3763514652252198, 2.4691238021850586, 1.2853179144859315, 1.089557173728943]
[0.004, 0.022, 0.034, 0.019, 0.039, 0.008, 0.07, 0.011, 0.049, 0.03]
[0.5159107534747622, 0.5898866130212144, 0.5726956839795172, 0.5823884418434528, 0.5943672275054864, 0.5820226773957571, 0.601865398683248, 0.5596196049743964, 0.6106437454279444, 0.6129297732260425]
